### Merging R parameter comparison tables

In [36]:
import os
import numpy as np
from matchms.importing import load_from_mgf
import pandas as pd
from itertools import compress

In [37]:
path_data = "/Users/krv114/Desktop/MS/Similarity comparison/conference_2023"
file_mgf_reference = os.path.join(path_data, "ex_spectra_stds_NEG_scans_50.mgf")
reference_spectra = list(load_from_mgf(file_mgf_reference))

In [38]:
file_mgf_query = os.path.join(path_data, "pseudo_query_neg_v1.mgf")
query_spectra = list(load_from_mgf(file_mgf_query))

Set up melt df with compound name_collision energy for the reference spectra and scan id_collision energy for the query

In [39]:
res = []

dir_path = path_data + '/parameter_test_NEG'

# Iterate directory
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)

In [40]:
ref_spec_feature_ids = [s.metadata['feature_id'] for s in reference_spectra]
query_spec_scan_ids = [s.metadata['scanindex'] for s in query_spectra]

In [41]:
out_df = pd.DataFrame()
for i, res_file in enumerate(res):
    new_colnames = []
    test_df = pd.read_csv(os.path.join(dir_path, res_file), sep=',')
    for feature_id in list(test_df.columns)[:-1]:
        ref_spec = reference_spectra[ref_spec_feature_ids.index(feature_id)]
        new_colnames.append(ref_spec.metadata['compound_name'] + '_' + ref_spec.metadata['collision_energy'])
    test_df.columns = new_colnames + ['id_query']
    new_rownames = []
    for scan_index in list(test_df['id_query'].values):
        query_spec = query_spectra[query_spec_scan_ids.index(str(scan_index))]
        new_rownames.append(query_spec.metadata['scanindex'] + '_' + query_spec.metadata['collision_energy'])
    test_df['id_query'] = new_rownames
    #melt_df = pd.melt(test_df, id_vars=test_df.columns[-1], var_name='id_query', value_name=res_file)
    melt_df = pd.melt(test_df, id_vars='id_query', value_name=res_file, var_name='ref_id')
    if i == 0:
        out_df = melt_df
    else:
        out_df[res_file] = list(melt_df[res_file].values)

In [ ]:
out_df.to_csv(os.path.join(path_data, 'parameter_tests_NEG.csv'), index=False)

In [ ]:
out_df = pd.read_csv(os.path.join(path_data, 'parameter_tests_NEG.csv'))

In [ ]:
out_df

,id_query,ref_id,scores_nspecangle_m_0_n_1_tolerance_0.01.csv,scores_nspecangle_m_0.5_n_1_tolerance_0.005.csv,scores_cosine_m_0_n_1_tolerance_0.01.csv,scores_nspecangle_m_0.5_n_0_tolerance_0.005.csv,scores_cosine_m_1_n_0.5_tolerance_0.005.csv,scores_cosine_m_1_n_2_tolerance_0.005.csv,scores_navdis_m_2_n_2_tolerance_0.005.csv,scores_navdis_m_2_n_1_tolerance_0.005.csv,...,scores_cosine_m_0_n_0_tolerance_0.01.csv,scores_neuclidean_m_0_n_0_tolerance_0.005.csv,scores_nspecangle_m_0_n_0.5_tolerance_0.01.csv,scores_cosine_m_0_n_0.5_tolerance_0.005.csv,scores_neuclidean_m_2_n_1_tolerance_0.01.csv,scores_neuclidean_m_0.5_n_2_tolerance_0.01.csv,scores_navdis_m_0_n_0.5_tolerance_0.01.csv,scores_neuclidean_m_1_n_2_tolerance_0.01.csv,scores_navdis_m_0.5_n_0.5_tolerance_0.005.csv,scores_neuclidean_m_0_n_1_tolerance_0.005.csv
0,1094_10,(2-Aminoethyl)Phosphonate_10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1,1,0.000000,0.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000
1,1095_20,(2-Aminoethyl)Phosphonate_10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1,1,0.000000,0.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000
2,1039_30,(2-Aminoethyl)Phosphonate_10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1,1,0.000000,0.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000
3,1091_40,(2-Aminoethyl)Phosphonate_10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1,1,0.000000,0.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000
4,1098_60,(2-Aminoethyl)Phosphonate_10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1,1,0.000000,0.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4693675,4758_10,Gamma-Muricholic Acid_60,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1,1,0.000000,0.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000
4693676,4759_20,Gamma-Muricholic Acid_60,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1,1,0.000000,0.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000
4693677,4760_30,Gamma-Muricholic Acid_60,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1,1,0.000000,0.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000
4693678,4761_40,Gamma-Muricholic Acid_60,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1,1,0.000000,0.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000


In [ ]:
keep_row = []
for a, b in zip(out_df['id_query'].values, out_df['ref_id'].values):
    if a[-2:] == b[-2:]:
        keep_row.append(True)
    else:
        keep_row.append(False)
out_df = out_df[keep_row].reset_index(drop=True)

In [ ]:
cols_to_keep = [False, False]
for col in list(out_df.columns)[2:]:
    cols_to_keep.append(all(out_df[col].values == 0) or all(out_df[col].values == 1))
cols_to_keep = [not elem for elem in cols_to_keep]

In [ ]:
out_df_filtered = out_df[list(compress(list(out_df.columns), cols_to_keep))]

In [ ]:
out_df_filtered.to_csv(os.path.join(path_data, 'parameter_tests_NEG_filtered.csv'), index=False)

In [ ]:
out_df_filtered

,id_query,ref_id,scores_nspecangle_m_0_n_1_tolerance_0.01.csv,scores_nspecangle_m_0.5_n_1_tolerance_0.005.csv,scores_cosine_m_0_n_1_tolerance_0.01.csv,scores_nspecangle_m_0.5_n_0_tolerance_0.005.csv,scores_cosine_m_1_n_0.5_tolerance_0.005.csv,scores_cosine_m_1_n_2_tolerance_0.005.csv,scores_navdis_m_2_n_2_tolerance_0.005.csv,scores_navdis_m_2_n_1_tolerance_0.005.csv,...,scores_neuclidean_m_0_n_2_tolerance_0.005.csv,scores_navdis_m_0_n_2_tolerance_0.01.csv,scores_nspecangle_m_0_n_0.5_tolerance_0.01.csv,scores_cosine_m_0_n_0.5_tolerance_0.005.csv,scores_neuclidean_m_2_n_1_tolerance_0.01.csv,scores_neuclidean_m_0.5_n_2_tolerance_0.01.csv,scores_navdis_m_0_n_0.5_tolerance_0.01.csv,scores_neuclidean_m_1_n_2_tolerance_0.01.csv,scores_navdis_m_0.5_n_0.5_tolerance_0.005.csv,scores_neuclidean_m_0_n_1_tolerance_0.005.csv
0,1094_10,(2-Aminoethyl)Phosphonate_10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1.000000,1.00000,0.000000,0.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000
1,1094_10,(2-Aminoethyl)Phosphonate_10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1.000000,1.00000,0.000000,0.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000
2,1094_10,(2-Aminoethyl)Phosphonate_10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1.000000,1.00000,0.000000,0.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000
3,1094_10,(2-Aminoethyl)Phosphonate_10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1.000000,1.00000,0.000000,0.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000
4,734_10,(2-Aminoethyl)Phosphonate_10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1.000000,1.00000,0.000000,0.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938731,4540_60,Gamma-Muricholic Acid_60,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1.000000,1.00000,0.000000,0.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000
938732,4426_60,Gamma-Muricholic Acid_60,7.735742e-01,7.941293e-01,9.374138e-01,0.321149,0.841593,9.971566e-01,0.657481,0.710606,...,0.808385,0.65559,0.514882,0.723442,0.889778,0.8084,0.818039,0.808383,0.812880,0.882662
938733,5325_60,Gamma-Muricholic Acid_60,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1.000000,1.00000,0.000000,0.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000
938734,5341_60,Gamma-Muricholic Acid_60,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,1.000000,1.000000,...,1.000000,1.00000,0.000000,0.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000
